# 30) Parallel reductions with CUDA.jl

Last time:
- Memory management with CUDA.jl
- Simple kernels using global memory  
- Simple kernels using shared/local memory  
- Instruction Level Parallelism  
-  Bank Conflicts

Today:

1. Outline
2. Parallel reduction on the GPU
3. Different strategies for optimization

## 1. Outline

In this lecture, we will use an efficient parallel reduction on the GPU as an example to talk about:

- communication across thread blocks
- global synchronization problem across thread blocks
- use of different memory pools for optimization strategies

:::{note} References:
- We will losely follow [Optimizing Parallel Reduction in CUDA](http://developer.download.nvidia.com/assets/cuda/files/reduction.pdf) by Mark Harris. Note that some of our kernels will be different than those given in the talk, but the ideas are (mostly) the same.
:::

## 2. Parallel reduction on the GPU

We want to perform a Parallel Reduction operation. 

This is a common and important data parallel primitive. It should be:

- Easy to implement in CUDA
- But often harder to get it right
- We can use this as a great optimization example

### Basic idea

- Tree-based approach used within each thread block

![A tree-based approach for a parallel reduction](../img/tree_based_parallel_reduction.png "A tree-based approach for a parallel reduction")

- Need to be able to use multiple thread blocks
  * to process very large arrays
  * to keep all multiprocessors on the GPU busy
  * each thread block reduces a portion of the array
- But how do we communicate partial results between
thread blocks?

### Problem: Global Synchronization

- If we could synchronize across all thread blocks, we _could_ easily reduce very large arrays:
  * global sync after each block would produce its result
  * once all blocks reach sync, we would continue recursively
- But **CUDA has no global synchronization.** Why?
  * It's expensive to build in hardware for GPUs with high processor count
  * It would force programmers to run fewer blocks (no more than # multiprocessors $\times$ # resident blocks / multiprocessor) to avoid deadlock, which may reduce overall efficiency 

**Solution**:  decompose into multiple kernels.

- Kernel launch serves as a global synchronization point
- Kernel launch has negligible hardware overhead and low software overhead

### Solution: Kernel Decomposition

- We avoid global sync by decomposing the computation into multiple kernel invocations


![A kernel decomposition for a parallel reduction on the GPU](../img/kernel_decomposition_for_reduction.png "A kernel decomposition for a parallel reduction on the GPU")

- In the case of reductions, the code for all levels is the
same: 
  * we have _recursive_ kernel invocations

### Measuring success: what are we striving for?

- We should strive to reach GPU peak performance

- But we need to choose the right metric to measure success:
  * GFLOP/s: for compute-bound kernels
  * Bandwidth: for memory-bound kernels

- Reductions have very low arithmetic intensity
  * 1 flop per element loaded (bandwidth-optimal)
- Therefore we are _not_ compute-bound, hence we should strive for peak bandwidth